# **Traitement des Données**

## **Filtrage des Commentaires Négatifs**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Path to kaggle CSV file
file_path = '/content/drive/My Drive/AI_project_dataset/Reviews.csv'

# Load the dataset
reviews_df = pd.read_csv(file_path, engine='python')

In [ ]:

# Keep only interesting columns
reviews_df = reviews_df[['UserId', 'ProductId', 'Score', 'Summary', 'Text']]

# Keep scores between 1 and 2 stars
negative_reviews = reviews_df[reviews_df['Score'] <= 2]
print(negative_reviews.head())
print("We have found",len(negative_reviews), "negative reviews over", len(reviews_df), "total reviews")

            UserId   ProductId  Score                               Summary  \
1   A1D87F6ZCVE5NK  B00813GRG4      1                     Not as Advertised   
3   A395BORC6FGVXV  B000UA0QIQ      2                        Cough Medicine   
12   A327PCT23YH90  B0009XLVG0      1  My Cats Are Not Fans of the New Food   
16  A3KLWF6WQ5BNYO  B001GVISJM      2                            poor taste   
26  A3RXAU2N8KV45G  B001GVISJM      1                       Nasty No flavor   

                                                 Text  
1   Product arrived labeled as Jumbo Salted Peanut...  
3   If you are looking for the secret ingredient i...  
12  My cats have been happily eating Felidae Plati...  
16  I love eating them and they are good for watch...  
26  The candy is just red , No flavor . Just  plan...  
We have found 82037 negative reviews over 568454 total reviews


## **Get only first 5 rows**

In [ ]:
#negative_reviews = negative_reviews.head()

## **Cleaning the text**




In [4]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import spacy
from tqdm import tqdm

# Download stopwords (à faire une seule fois)
nltk.download('stopwords')
nltk.download('punkt')

# Initialize spaCy model
nlp = spacy.load("en_core_web_sm")
# Cleaning text function
def clean_text(text):
    text = text.lower()  # Convertir en minuscule
    text = re.sub(r'\d+', '', text)  # Retirer les chiffres
    text = re.sub(r'[^\w\s]', '', text)  # Retirer les ponctuations
    text = re.sub(r'\s+', ' ', text).strip()  # Retirer les espaces multiples
    # Process the text with spaCy
    doc = nlp(text)
    #print("doc", doc)
    # Extract lemmas and return as a single string
    text = " ".join([token.lemma_ for token in doc])
    #print("lemmatized_text", text)
    return text

#negative_reviews['lemmatized_text'] = negative_reviews['Text'].apply(clean_text)
#negative_reviews['Cleaned_Summary'] = negative_reviews['Summary'].apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## **Tokenization**

In [ ]:
# Tokenization
negative_reviews['Tokens'] = negative_reviews['lemmatized_text'].apply(word_tokenize)


<ipython-input-30-a1fcfb76bb86>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_reviews['Tokens'] = negative_reviews['lemmatized_text'].apply(word_tokenize)


In [ ]:
negative_reviews['Tokens']

1         [product, arrive, label, as, jumbo, salt, pean...
3         [if, you, be, look, for, the, secret, ingredie...
12        [my, cat, have, be, happily, eat, felidae, pla...
16        [I, love, eat, they, and, they, be, good, for,...
26        [the, candy, be, just, red, no, flavor, just, ...
                                ...                        
568433    [I, just, buy, this, soup, today, at, my, loca...
568434    [this, soup, be, mostly, broth, although, it, ...
568435    [it, be, mostly, broth, with, the, advertised,...
568446    [I, have, order, some, of, these, a, few, mont...
568450    [I, m, disappointed, with, the, flavor, the, c...
Name: Tokens, Length: 82037, dtype: object

## **Delete Stop Words**

In [ ]:

# Retirer les mots vides (stopwords)
stop_words = set(stopwords.words('english'))  # Utiliser 'french' pour le français
negative_reviews['Tokens'] = negative_reviews['Tokens'].apply(lambda x: [word for word in x if word not in stop_words])


<ipython-input-34-e1ed51d99cd2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_reviews['Tokens'] = negative_reviews['Tokens'].apply(lambda x: [word for word in x if word not in stop_words])


In [ ]:
negative_reviews['Tokens']

1         [product, arrive, label, jumbo, salt, peanutst...
3         [look, secret, ingredient, robitussin, I, beli...
12        [cat, happily, eat, felidae, platinum, two, ye...
16        [I, love, eat, good, watch, tv, look, movie, s...
26        [candy, red, flavor, plan, chewy, I, would, ne...
                                ...                        
568433    [I, buy, soup, today, local, grocer, bring, ba...
568434    [soup, mostly, broth, although, kick, also, sw...
568435    [mostly, broth, advertised, cup, vegetable, ho...
568446    [I, order, month, back, great, late, batch, te...
568450    [I, disappointed, flavor, chocolate, note, esp...
Name: Tokens, Length: 82037, dtype: object

## **Remove duplicates**

In [ ]:
# Check missing data
print(negative_reviews.isnull().sum())


UserId              0
ProductId           0
Score               0
Summary            25
Text                0
lemmatized_text     0
Tokens              0
dtype: int64


In [ ]:

# Check duplicates
duplicate_rows = negative_reviews[negative_reviews.duplicated(subset=['lemmatized_text','ProductId'])]
print(f"Number of duplicate rows: {duplicate_rows.shape[0]}")



Number of duplicate rows: 347


In [ ]:
# Delete duplicates
negative_reviews = negative_reviews.drop_duplicates(subset=['lemmatized_text'])
if duplicate_rows.shape[0] == 0:
  print("No duplicates where found")
else:
  print(f"Number of rows after removing duplicates: {negative_reviews.shape[0]}")

                UserId   ProductId  Score  \
2647    A2NLZ3M0OJV9NX  B0016FY6H6      2   
2946    A2ISKAWUPGGOLZ  B0002TJAZK      1   
2947    A3TVZM3ZIXG8YW  B0002TJAZK      1   
5929     A3L27V7TOG1JZ  B001O2IX8E      1   
5958    A3KDZCQ82JFWLN  B001O2IX8E      1   
...                ...         ...    ...   
539358   AF3BYMPWKWO8F  B001BDDT8K      1   
539359   AF3BYMPWKWO8F  B001BDDT8K      1   
555968  A1RR835I8VF0VV  B005DGIAUI      1   
558042  A2RNYW2HIQBGSM  B000EM9E2Y      1   
561051  A2NLZ3M0OJV9NX  B0012BSMHY      2   

                                                  Summary  \
2647                           way too weak for my tastes   
2946                                      Kitty Junk Food   
2947    Filler food is empty, leaves your cat always n...   
5929            careful, moldy product, USDA now involved   
5958                                     Some broken jars   
...                                                   ...   
539358                        Wa


## **Save csv**

In [ ]:
# Assume 'negative_reviews' is your DataFrame
file_path = '/content/drive/My Drive/AI_project/dataset/negative_reviews.csv'
negative_reviews.to_csv(file_path, index=False)

NameError: name 'negative_reviews' is not defined

##**Load data precleaned**

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Path to your CSV file
file_path = '/content/drive/My Drive/AI_project/dataset/negative_reviews.csv'

# Load the dataset
clean_df = pd.read_csv(file_path, engine='python')

Mounted at /content/drive


## **Label data using GPT3**

In [11]:
!pip install openai

In [12]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
# Path to your CSV file
file_path = '/content/drive/My Drive/AI_project/dataset/negative_reviews_topics.csv'
# Load the dataset
clean_df = pd.read_csv(file_path, engine='python')
clean_df
df=clean_df.head(2000)
#df=clean_df
len(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2000

In [17]:
import pandas as pd
import openai
from tqdm import tqdm
import time
import backoff
import random

openai.api_key = "enter your api"

@backoff.on_exception(backoff.expo, openai.error.RateLimitError, max_tries=10, factor=2)
def get_labels_batch(texts):
    prompt = (
        "Assign a single-word label from the following list (size, taste, quantity, price, quality, other) to each of the following texts. "
        "Here are a few examples: "
        'Example1: "If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The flavor is very medicinal." Label: taste; '
        'Example2: "This oatmeal is not good. It\'s mushy, soft, I don\'t like it. Quaker Oats is the way to go." Label: quality; '
        'Example3: "Hey, the description says 360 grams - that is roughly 13 ounces at under $4.00 per can. No way - that is the approximate price for a 100 gram can." Label: price; '
        'Example4: "I felt energized within five minutes, but it lasted for about 45 minutes. I paid $3.99 for this drink. I could have just drunk a cup of coffee and saved my money." Label: quality; '
    )

    for i, text in enumerate(texts):
        prompt += f'Text {i+1}: "{text}" Label: '

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that labels text."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=10 * len(texts)
    )

    labels = response.choices[0].message['content'].strip().split('\n')

    # Ensure we have the correct number of labels
    if len(labels) != len(texts):
        print(f"Warning: Expected {len(texts)} labels, but got {len(labels)}. Retrying batch.")
        time.sleep(random.uniform(1, 3))  # Add random delay to avoid rate limit
        return get_labels_batch(texts)

    return [label.split(':')[-1].strip().lower() for label in labels]

# Assuming df is already defined and contains the 'lemmatized_text' column
# df = pd.read_csv('path_to_your_csv')  # Uncomment and adjust this line if df is not already loaded

# Apply the function to each text entry in the DataFrame using batching
batch_size = 1  # Reduce batch size to decrease token usage per request
texts = df['lemmatized_text'].tolist()
labels = []

start_time = time.time()

for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i:i + batch_size]
    batch_labels = get_labels_batch(batch_texts)

    # Debugging: print the length of batch_labels and check if it matches the batch size
    print(f"Processed batch {i // batch_size + 1}, Batch size: {len(batch_texts)}, Labels returned: {len(batch_labels)}")

    labels.extend(batch_labels)

end_time = time.time()
total_time = end_time - start_time

df['label'] = labels

# Debugging: Check the length of the final labels list
print(f"Total labels: {len(labels)}, Expected: {len(texts)}")

# Print the DataFrame with assigned labels
print(df)

# Save the DataFrame to a CSV file
file_path = '/content/drive/My Drive/AI_project/dataset/labeled_reviews2000.csv'
df.to_csv(file_path, index=False)

print(f"Total time taken: {total_time:.2f} seconds")


  0%|          | 1/2000 [00:00<22:36,  1.47it/s]

Processed batch 1, Batch size: 1, Labels returned: 1


  0%|          | 2/2000 [00:01<27:17,  1.22it/s]

Processed batch 2, Batch size: 1, Labels returned: 1


  0%|          | 3/2000 [00:02<22:56,  1.45it/s]

...

100%|█████████▉| 1999/2000 [1:19:08<00:01,  1.84s/it]INFO:backoff:Backing off get_labels_batch(...) for 0.3s (openai.error.RateLimitError: Rate limit reached for gpt-4 in organization org-BWZXek8IoyBebWyrPhEhUphD on tokens per min (TPM): Limit 10000, Used 9719, Requested 482. Please try again in 1.206s. Visit https://platform.openai.com/account/rate-limits to learn more.)


Processed batch 1999, Batch size: 1, Labels returned: 1


INFO:backoff:Backing off get_labels_batch(...) for 4.0s (openai.error.RateLimitError: Rate limit reached for gpt-4 in organization org-BWZXek8IoyBebWyrPhEhUphD on tokens per min (TPM): Limit 10000, Used 9649, Requested 482. Please try again in 786ms. Visit https://platform.openai.com/account/rate-limits to learn more.)
100%|██████████| 2000/2000 [1:19:13<00:00,  2.38s/it]

Processed batch 2000, Batch size: 1, Labels returned: 1
Total labels: 2000, Expected: 2000
              UserId   ProductId  Score  \
0     A1D87F6ZCVE5NK  B00813GRG4      1   
1     A395BORC6FGVXV  B000UA0QIQ      2   
2      A327PCT23YH90  B0009XLVG0      1   
3     A3KLWF6WQ5BNYO  B001GVISJM      2   
4     A3RXAU2N8KV45G  B001GVISJM      1   
...              ...         ...    ...   
1995  A3P5T6X3X1HCPI  B00513J4D0      1   
1996  A3QNQQKJTL76H0  B004H4NC4Q      2   
1997  A2GINYK25G77M2  B003F8XZSS      1   
1998  A1UECUMTG3VWOL  B000AY9U20      2   
1999  A3FICOMPE2KFGY  B000AY9U20      1   

                                                Summary  \
0                                     Not as Advertised   
1                                        Cough Medicine   
2                  My Cats Are Not Fans of the New Food   
3                                            poor taste   
4                                       Nasty No flavor   
...                                   


<ipython-input-17-eda9d7d5e31b>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = labels


## **Label data using GPT4**

In [9]:
!pip install backoff

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [10]:
import pandas as pd
import openai
from tqdm import tqdm
import time
import backoff

openai.api_key = "enter your api"

@backoff.on_exception(backoff.expo, openai.error.RateLimitError, max_tries=10)
def get_labels_batch(texts):
    prompt = (
        "Assign a single-word label from the following list (size, taste, quantity, price, quality, other) to each of the following texts. "
        "Here are a few examples: "
        'Example1: "If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The flavor is very medicinal." Label: taste; '
        'Example2: "This oatmeal is not good. It\'s mushy, soft, I don\'t like it. Quaker Oats is the way to go." Label: quality; '
        'Example3: "Hey, the description says 360 grams - that is roughly 13 ounces at under $4.00 per can. No way - that is the approximate price for a 100 gram can." Label: price; '
        'Example4: "I felt energized within five minutes, but it lasted for about 45 minutes. I paid $3.99 for this drink. I could have just drunk a cup of coffee and saved my money." Label: quality; '
    )

    for i, text in enumerate(texts):
        prompt += f'Text {i+1}: "{text}" Label: '

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that labels text."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=10 * len(texts)
    )

    labels = response.choices[0].message['content'].strip().split('\n')
    return [label.split(':')[-1].strip().lower() for label in labels]

# Assuming df is already defined and contains the 'lemmatized_text' column
# df = pd.read_csv('path_to_your_csv')  # Uncomment and adjust this line if df is not already loaded

# Apply the function to each text entry in the DataFrame using batching
batch_size = 10
texts = df['lemmatized_text'].tolist()
labels = []

start_time = time.time()

for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i:i + batch_size]
    batch_labels = get_labels_batch(batch_texts)
    labels.extend(batch_labels)

end_time = time.time()
total_time = end_time - start_time

df['label'] = labels

# Print the DataFrame with assigned labels
print(df)

# Save the DataFrame to a CSV file
file_path = '/content/drive/My Drive/AI_project/dataset/labeled_reviews2000.csv'
df.to_csv(file_path, index=False)

print(f"Total time taken: {total_time:.2f} seconds")


ModuleNotFoundError: No module named 'openai'

In [ ]:
len(labels)

In [ ]:
#!pip install openai==0.28

In [ ]:
file_path = '/content/drive/My Drive/AI_project/dataset/labeled_reviews2.csv'
df2 = pd.read_csv(file_path, engine='python')
print(df2)

In [ ]:
# Encode labels (0 for negative, 1 for positive)
df['label'] = df['label'].str.lower()
label_dict = {"other": 0, "size": 1, "taste": 2, "quantity": 3, "quality": 4, "price": 5}
df['label_numerical'] = df['label'].map(label_dict)
print(df["label"].unique())

In [ ]:
# Get summary statistics for numeric columns
df["ProductId"].nunique()


## **BertCLassifier**

In [2]:

import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

#Load data
# Path to your CSV file
file_path = '/content/drive/My Drive/AI_project/dataset/labeled_reviews2000.csv'

# Load the dataset
df = pd.read_csv(file_path, engine='python')

# Example DataFrame
#df = df.head(500)
#df = clean_df.head(1000)


# Step 1: Print unique values of the 'category' column
unique_values = df['label'].unique()
print("Unique values (original):", unique_values)

# Step 2: Convert them to lowercase
unique_values_lower = [value.lower() for value in unique_values]
print("Unique values (lowercase):", unique_values_lower)


Mounted at /content/drive
Unique values (original): ['size' 'taste' 'quality' 'other' 'price' 'quantity']
Unique values (lowercase): ['size', 'taste', 'quality', 'other', 'price', 'quantity']


In [5]:
# Encode labels
df['label']  = df['label'] .apply(clean_text)
df['label'] = df['label'].str.lower()
label_dict = {"other": 0, "size": 1, "taste": 2, "quantity": 3, "quality": 4, "price": 5}
df['label_numerical'] = df['label'].map(label_dict)
print(df["label"].unique())

['size' 'taste' 'quality' 'other' 'price' 'quantity']


In [6]:
print(df["label_numerical"].unique())

[1 2 4 0 5 3]


### **Train the whole model**

In [8]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from torch.nn.utils.rnn import pad_sequence


# Encode labels (0 for negative, 1 for positive)
"""df['label'] = df['label'].str.lower()
label_dict = {"other": 0, "size": 1, "taste": 2, "quantity": 3, "quality": 4, "price": 5}
df['label_numerical'] = df['label'].map(label_dict)
print(df["label_numerical"])"""

# Custom dataset class
class FeedbackDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label_numerical': torch.tensor(label, dtype=torch.long)
        }

# Initialize the tokenizer
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
#tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
#tokenizer = BertTokenizer.from_pretrained('bert-large-cased')

# Initialize the model
#model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=6)
#model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=6)
#model = BertForSequenceClassification.from_pretrained('bert-large-cased', num_labels=6)

# Create dataset
texts = df['lemmatized_text'].tolist()
#texts = df['Text'].tolist()
labels = df['label_numerical'].tolist()

# Split dataset into train and eval (using 80-20 split)
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Create custom dataset instances
train_dataset = FeedbackDataset(texts=train_texts, labels=train_labels, tokenizer=tokenizer, max_len=128)
val_dataset = FeedbackDataset(texts=val_texts, labels=val_labels, tokenizer=tokenizer, max_len=128)

# Custom collate function for padding
def custom_collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['label_numerical'] for item in batch]

    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = torch.stack(labels)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'label_numerical': labels
    }
#The DataLoader is a PyTorch class that provides an iterable over a dataset, handling the batching, shuffling, and loading of data.
# Create DataLoaders with custom collate function
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=custom_collate_fn)



device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=0.00001)

# Training function
def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label_numerical'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    return total_loss / len(data_loader)

# Early stopping class
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# Evaluation function
def eval_model(model, data_loader, device):
    model.eval()
    predictions, true_labels = [], []
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label_numerical'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    return predictions, true_labels, total_loss / len(data_loader)

# Training and evaluation loop
num_epochs = 4
early_stopping = EarlyStopping(patience=3, min_delta=0.001)

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, device)
    predictions, true_labels, val_loss = eval_model(model, val_loader, device)

    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping")
        break

# After training your model, save the model and tokenizer
model.save_pretrained('/content/drive/My Drive/AI_project/models/bert_large_uncased_2000')
tokenizer.save_pretrained('/content/drive/My Drive/AI_project/models/tokenizer_large_uncased_2000')

# Evaluate the model
predictions, true_labels, tl = eval_model(model, val_loader, device)

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted', pos_label=1)
recall = recall_score(true_labels, predictions, average='weighted', pos_label=1)
f1 = f1_score(true_labels, predictions, average='weighted', pos_label=1)
class_report = classification_report(true_labels, predictions)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print('Classification Report:')
print(class_report)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4, Training Loss: 1.0301, Validation Loss: 0.9577
Epoch 2/4, Training Loss: 0.6337, Validation Loss: 0.7049
Epoch 3/4, Training Loss: 0.3498, Validation Loss: 0.6352
Epoch 4/4, Training Loss: 0.1770, Validation Loss: 0.6305
Accuracy: 0.8100
Precision: 0.8149
Recall: 0.8100
F1 Score: 0.8051
Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.52      0.58        21
           1       0.75      0.75      0.75         8
           2       0.83      0.91      0.87       139
           3       0.80      0.29      0.42        14
           4       0.85      0.80      0.83       189
           5       0.62      0.90      0.73        29

    accuracy                           0.81       400
   macro avg       0.75      0.69      0.70       400
weighted avg       0.81      0.81      0.81       400



### **Train the 2 last layers**

In [8]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from torch.nn.utils.rnn import pad_sequence

# Custom dataset class
class FeedbackDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label_numerical': torch.tensor(label, dtype=torch.long)
        }

# Initialize the tokenizer
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
#tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
#tokenizer = BertTokenizer.from_pretrained('bert-large-cased')

# Initialize the model
#model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=6)
#model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=6)
#model = BertForSequenceClassification.from_pretrained('bert-large-cased', num_labels=6)

# Freeze all layers except the last two
for name, param in model.named_parameters():
    if 'encoder.layer' in name and name.split('.')[2].isdigit():
        layer_num = int(name.split('.')[2])
        if layer_num < 22:  # Assuming BERT large has 24 layers
            param.requires_grad = False

# Create dataset
texts = df['Text'].tolist()
labels = df['label_numerical'].tolist()

# Split dataset into train and eval (using 80-20 split)
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Create custom dataset instances
train_dataset = FeedbackDataset(texts=train_texts, labels=train_labels, tokenizer=tokenizer, max_len=128)
val_dataset = FeedbackDataset(texts=val_texts, labels=val_labels, tokenizer=tokenizer, max_len=128)

# Custom collate function for padding
def custom_collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['label_numerical'] for item in batch]

    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = torch.stack(labels)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'label_numerical': labels
    }

# Create DataLoaders with custom collate function
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=custom_collate_fn)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Optimizer
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.00001)

# Training function
def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label_numerical'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    return total_loss / len(data_loader)

# Early stopping class
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# Evaluation function
def eval_model(model, data_loader, device):
    model.eval()
    predictions, true_labels = [], []
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label_numerical'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    return predictions, true_labels, total_loss / len(data_loader)

# Training and evaluation loop
num_epochs = 20
early_stopping = EarlyStopping(patience=3, min_delta=0.001)

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, device)
    predictions, true_labels, val_loss = eval_model(model, val_loader, device)

    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping")
        break

# After training your model, save the model and tokenizer
model.save_pretrained('/content/drive/My Drive/AI_project/models/bert_large_uncased_2layer_500')
tokenizer.save_pretrained('/content/drive/My Drive/AI_project/models/tokenizer_large_cased_2layer_500')

# Evaluate the model
predictions, true_labels, tl = eval_model(model, val_loader, device)

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted', pos_label=1)
recall = recall_score(true_labels, predictions, average='weighted', pos_label=1)
f1 = f1_score(true_labels, predictions, average='weighted', pos_label=1)
class_report = classification_report(true_labels, predictions)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print('Classification Report:')
print(class_report)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20, Training Loss: 1.1339, Validation Loss: 1.0039
Epoch 2/20, Training Loss: 0.7929, Validation Loss: 0.7249
Epoch 3/20, Training Loss: 0.5245, Validation Loss: 0.6229
Epoch 4/20, Training Loss: 0.3140, Validation Loss: 0.7247
Epoch 5/20, Training Loss: 0.2023, Validation Loss: 0.9372
Epoch 6/20, Training Loss: 0.1245, Validation Loss: 0.7386
Early stopping
Accuracy: 0.7825
Precision: 0.7847
Recall: 0.7825
F1 Score: 0.7816
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.52      0.51        21
           1       1.00      0.50      0.67         8
           2       0.83      0.83      0.83       139
           3       0.38      0.36      0.37        14
           4       0.79      0.82      0.81       189
           5       0.85      0.76      0.80        29

    accuracy                           0.78       400
   macro avg       0.73      0.63      0.66       400
weighted avg       0.78      0.78      0.78       400

##**Load data and model to eval**

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import pickle
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Define the custom dataset class
class FeedbackDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label_numerical': torch.tensor(label, dtype=torch.long)
        }

# Load the dataset
with open('/content/drive/My Drive/AI_project/dataset/val_data.pkl', 'rb') as f:
    data, labels = pickle.load(f)

# Custom collate function for padding
def custom_collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['label_numerical'] for item in batch]

    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = torch.stack(labels)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'label_numerical': labels
    }

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('/content/drive/My Drive/AI_project/models/tokenizer_model6')

# Load the model
model = BertForSequenceClassification.from_pretrained('/content/drive/My Drive/AI_project/models/bert_label_model6')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Create the validation dataset and DataLoader
val_dataset = FeedbackDataset(data, labels, tokenizer=tokenizer, max_len=128)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=custom_collate_fn)

# Define the evaluation function
def eval_model(model, data_loader, device):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label_numerical'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    return predictions, true_labels

# Evaluate the model
predictions, true_labels = eval_model(model, val_loader, device)

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted', pos_label=1)
recall = recall_score(true_labels, predictions, average='weighted', pos_label=1)
f1 = f1_score(true_labels, predictions, average='weighted', pos_label=1)
class_report = classification_report(true_labels, predictions)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print('Classification Report:')
print(class_report)


## **Test GPT2 to generate product suggestion**





In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

#file_path = '/content/drive/My Drive/AI_project/dataset/labeled_reviews_example_gpt4.csv'
file_path = '/content/drive/My Drive/AI_project/dataset/labeled_reviews2000.csv'
df = pd.read_csv(file_path, engine='python')
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Ensure you have GPU support if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import torch
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Ensure you have GPU support if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained GPT-2 model and tokenizer
#model_name = "gpt2"
#model_name = "gpt2-large"
model_name = "gpt2-xl"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Move model to the appropriate device (GPU or CPU)
model.to(device)
"""# Assuming you have a DataFrame with 'lemmatized_text' and 'label' columns
data = {
    'lemmatized_text': [
        "If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The flavor is very medicinal.",
        "This oatmeal is not good. It's mushy, soft, I don't like it. Quaker Oats is the way to go.",
        "Hey, the description says 360 grams - that is roughly 13 ounces at under $4.00 per can. No way - that is the approximate price for a 100 gram can."
    ],
    'label': ['taste', 'quality', 'price']
}

df = pd.DataFrame(data)"""

# Example Data Collator and Dataset (replace with actual paths and variables)
# Here we skip the fine-tuning part for simplicity
# Fine-tuning should be done similarly to the previous code snippet if needed

def generate_improvement_suggestion(feedback, category, max_input_length=512, max_new_tokens=50):
    examples = [
        {
            "feedback": "If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The flavor is very medicinal.",
            "category": "taste",
            "suggestion": "Improve the taste of the soda by using natural ingredients."
        },
        {
            "feedback": "This oatmeal is not good. It's mushy, soft, I don't like it. Quaker Oats is the way to go.",
            "category": "quality",
            "suggestion": "Use high-quality whole grain oats that maintain a good texture when cooked."
        },
        {
            "feedback": "Hey, the description says 360 grams - that is roughly 13 ounces at under $4.00 per can. No way - that is the approximate price for a 100 gram can.",
            "category": "price",
            "suggestion": "Increase the product quantity or reduce the price to provide better value."
        }
    ]

    # Construct the prompt with examples
    prompt = "Here are some suggestions for improving the product quality based on negative customer feedback. Each suggestion should be one sentence and specific to the feedback category:\n\n"
    for example in examples:
        prompt += f"Feedback: \"{example['feedback']}\"\nCategory: {example['category']}\nSuggestion: {example['suggestion']}\n\n"

    # Add the new feedback and category
    prompt += f"Feedback: \"{feedback}\"\nCategory: {category}\nSuggestion:"

    # Encode the input prompt
    inputs = tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=max_input_length).to(device)

    # Generate text
    outputs = model.generate(
        inputs,
        max_new_tokens=max_new_tokens,  # Limit the number of new tokens generated
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        early_stopping=True
    )

    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract and return the generated suggestion
    improvement_suggestion = generated_text.split("Suggestion:")[-1].strip().split('\n')[0]
    return improvement_suggestion
suggestions = []
# Example usage with DataFrame
for index, row in df.iterrows():
    feedback = row['lemmatized_text']
    category = row['label']
    suggestion = generate_improvement_suggestion(feedback, category)
    print(f"Feedback: {feedback}\nCategory: {category}\nGenerated Suggestion: {suggestion}\n")
    suggestions.append(suggestion)

# Add the suggestions to the DataFrame
df['suggestions'] = suggestions


